# Barcode per rabies cell

We have 2 hybridisation probes: one for all rabies genes and one for the padlocks.
Detect cells using the hybridisation probes and then count the number of rabies padlocks in each cell.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import iss_preprocess as issp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from iss_preprocess.io import load_ops, get_roi_dimensions
from iss_preprocess.pipeline.register import load_and_register_tile

In [ ]:
project = "becalia_rabies_barseq"
mouse = "BRAC8780.3f"
chamber = "chamber_08"
prefix = "hybridisation_round_1_1"

In [ ]:
data_path = f"{project}/{mouse}/{chamber}"
print(data_path)
ops = issp.io.load_ops(data_path)
roi_dims = issp.io.load.get_roi_dimensions(data_path, prefix=prefix)

In [ ]:
example_tile, bad_px = load_and_register_tile(
    data_path,
    prefix=prefix,
    tile_coors=(10, 0, 0),
    projection="max-median",
    correct_illumination=False,
    filter_r=False,
)
# keep only round 1
example_tile = example_tile[..., 0]
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
for iax, ax in enumerate(axes.flat):
    vmax = np.percentile(example_tile[..., iax], 99)
    ax.imshow(example_tile[..., iax], interpolation="none", cmap="inferno", vmax=vmax)
    ax.set_title(f"Channel {iax} - vmax {int(vmax)}")
    ax.set_axis_off()
fig.tight_layout()
print(example_tile.shape)

In [ ]:
rab_genes_channel = 1
padlock_channel = 3
chans = [rab_genes_channel, padlock_channel]
colors = [(1, 0, 0), (0, 1, 0)]
vmaxs = [500, 200]
print(example_tile[..., chans].shape)
rgb = issp.vis.to_rgb(example_tile[..., chans], colors=colors, vmin=[0, 0], vmax=vmaxs)
plt.figure(figsize=(10, 10))
plt.imshow(rgb)
plt.axis("off")

In [ ]:
import cv2
from skimage.segmentation import expand_labels
from skimage import measure


def detect_rab_cells(
    stack,
    rab_genes_channel=1,
    padlock_channel=3,
    r1=9,
    r2=41,
    threshold=100,
    footprint=10,
):
    rab_genes = stack[..., rab_genes_channel]
    chans = [rab_genes_channel, padlock_channel]
    binary = issp.pipeline.segment._filter_mcherry_masks(
        rab_genes, r1, r2, threshold, footprint=footprint
    )
    labeled_image = measure.label(binary)
    labeled_image = expand_labels(labeled_image, distance=footprint)
    labeled_image, props_df = issp.segment.cells.label_image(
        binary, example_tile[..., chans]
    )
    return labeled_image, props_df

In [ ]:
# Detect rabies cells from gene channel
r1 = 9
r2 = 37
footprint = 10
threshold = 100
labeled_image, props_df = detect_rab_cells(
    example_tile,
    rab_genes_channel=rab_genes_channel,
    padlock_channel=padlock_channel,
    r1=r1,
    r2=r2,
    threshold=threshold,
    footprint=footprint,
)

rab_genes = example_tile[..., rab_genes_channel]

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(rab_genes, cmap="Greys_r", vmax=1000, vmin=0)
ax.contour(
    labeled_image,
    levels=np.arange(labeled_image.max()) + 0.5,
    colors="g",
    linewidths=0.5,
)
lab = labeled_image.astype(float)
lab[lab == 0] = np.nan
ax.imshow(lab % 20, cmap="tab20", alpha=0.5)
ax.set_axis_off()
fig.tight_layout()

In [ ]:
# Detect rabies spots from padlock channel
th = 20

padlock = example_tile[..., padlock_channel]
filtered = issp.image.filter_stack(padlock, r1=2, r2=4)
spots = issp.segment.detect_spots(filtered, threshold=th, spot_size=2)


fig, axes = plt.subplots(1, 2, figsize=(15, 10))
issp.vis.plot_matrix_with_colorbar(
    filtered, ax=axes[0], cmap="coolwarm", vmax=th, vmin=-th
)
axes[1].imshow(padlock, cmap="Greys_r", vmax=200, vmin=0)
axes[1].scatter(
    spots.x, spots.y, c="none", edgecolors="g", s=100, marker="o", facecolors="none"
)
for ax in axes:
    ax.set_xlim(1000, 2000)
    ax.set_ylim(1000, 2000)
    ax.set_axis_off()

# Run on all tiles

Repeat on all rabies tiles.

In [ ]:
from skimage.segmentation import expand_labels

rab_cells, spots_dfs = [], []
rabies_stack = None
labeled_images = None
for roi in roi_dims[:, 0]:
    print(f"Processing roi {roi}")
    stack, bad_px = issp.pipeline.load_and_register_tile(
        data_path,
        prefix=prefix,
        tile_coors=(roi, 0, 0),
        projection="max-median",
        correct_illumination=False,
        filter_r=False,
    )
    stack = stack[..., 0]
    labeled_image, props_df = detect_rab_cells(
        stack,
        rab_genes_channel=rab_genes_channel,
        padlock_channel=padlock_channel,
        r1=r1,
        r2=r2,
        threshold=threshold,
        footprint=footprint,
    )
    labeled_image = expand_labels(labeled_image, int(5 / 0.23))
    padlock = stack[..., padlock_channel]
    filtered = issp.image.filter_stack(padlock, r1=2, r2=4)
    spots = issp.segment.detect_spots(filtered, threshold=th, spot_size=2)
    spots["mask_id"] = labeled_image[spots.y, spots.x]
    spots["roi"] = roi
    props_df["roi"] = roi

    rab_cells.append(props_df)
    spots_dfs.append(spots)
    if rabies_stack is None:
        rabies_stack = np.zeros((stack.shape[0], stack.shape[1], 2, len(roi_dims)))
        labeled_images = np.zeros((stack.shape[0], stack.shape[1], len(roi_dims)))
    rabies_stack[..., roi - 1] = stack[..., chans].copy()
    labeled_images[..., roi - 1] = labeled_image

In [ ]:
from skimage.segmentation import expand_labels
from iss_preprocess.pipeline.register import load_and_register_tile


def detect_rab_cells(
    stack,
    rab_genes_channel=1,
    padlock_channel=3,
    r1=9,
    r2=41,
    threshold=100,
    footprint=10,
):
    rab_genes = stack[..., rab_genes_channel]
    chans = [rab_genes_channel, padlock_channel]
    binary = issp.pipeline.segment._filter_mcherry_masks(
        rab_genes, r1, r2, threshold, footprint=footprint
    )
    labeled_image, props_df = issp.segment.cells.label_image(binary, stack[..., chans])
    return labeled_image, props_df


rab_cells_new, spots_dfs_new = [], []
rabies_stack = None
labeled_images = None
for roi in roi_dims[:, 0]:
    print(f"Processing roi {roi}")
    stack, bad_px = load_and_register_tile(
        data_path,
        prefix=prefix,
        tile_coors=(roi, 0, 0),
        projection="max-median",
        correct_illumination=False,
        filter_r=False,
    )
    stack = stack[..., 0]
    labeled_image, props_df = detect_rab_cells(
        stack,
        rab_genes_channel=rab_genes_channel,
        padlock_channel=padlock_channel,
        r1=r1,
        r2=r2,
        threshold=threshold,
        footprint=footprint,
    )
    labeled_image = expand_labels(labeled_image, int(5 / 0.23))
    padlock = stack[..., padlock_channel]
    filtered = issp.image.filter_stack(padlock, r1=2, r2=4)
    spots = issp.segment.detect_spots(filtered, threshold=th, spot_size=2)
    spots["mask_id"] = labeled_image[spots.y, spots.x]
    spots["roi"] = roi
    props_df["roi"] = roi

    rab_cells_new.append(props_df)
    spots_dfs_new.append(spots)
    if rabies_stack is None:
        rabies_stack = np.zeros((stack.shape[0], stack.shape[1], 2, len(roi_dims)))
        labeled_images = np.zeros((stack.shape[0], stack.shape[1], len(roi_dims)))
    rabies_stack[..., roi - 1] = stack[..., chans].copy()
    labeled_images[..., roi - 1] = labeled_image

In [ ]:
plt.imshow(labeled_image)

In [ ]:
rab_cells = pd.concat(rab_cells_new)
spots_df = pd.concat(spots_dfs_new)
ops = issp.io.load_ops(data_path)
good_cells = issp.pipeline.segment._filter_masks(
    ops, rab_cells, labeled_image=None
).copy()

In [ ]:
good_cells["cell_uid"] = (
    good_cells["roi"].astype(str) + "_" + good_cells["label"].astype(str)
)
spots_df["cell_uid"] = (
    spots_df["roi"].astype(str) + "_" + spots_df["mask_id"].astype(str)
)
spots_df["valid_cell"] = spots_df["cell_uid"].isin(good_cells["cell_uid"])
spots_df["valid_cell"].value_counts()

In [ ]:
good_cells["cell_uid"] = (
    good_cells["roi"].astype(str) + "_" + good_cells["label"].astype(str)
)
spots_df["cell_uid"] = (
    spots_df["roi"].astype(str) + "_" + spots_df["mask_id"].astype(str)
)
spots_df["valid_cell"] = spots_df["cell_uid"].isin(good_cells["cell_uid"])
spots_df["valid_cell"].value_counts()

print(f"Number of cells: {len(good_cells)}")
good_cells["spot_count"] = 0
for cell_uid, count in spots_df["cell_uid"].value_counts().items():
    good_cells.loc[good_cells["cell_uid"] == cell_uid, "spot_count"] = count


fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].hist(good_cells["spot_count"], bins=np.arange(40) - 0.5)
perc, bins, bars = axes[1].hist(
    good_cells["spot_count"],
    bins=np.arange(50) - 0.5,
    cumulative=True,
    density=True,
)


for n in [1, 3, 5]:
    lt_n = good_cells["spot_count"].lt(n).sum()
    print(f"Cells with less than {n} spots: {lt_n}, {lt_n/len(good_cells)*100:.2f}%")
axes[0].set_xlabel("Barcode spots per cell")
axes[1].set_xlabel("Barcode spots per cell")
axes[0].set_ylabel("# of Cells")
axes[1].set_ylabel("Cumulative fraction")
axes[1].set_xlim(0, 20)
axes[1].set_ylim(0, 1)
axes[0].set_xlim(-0.5, 40)
fig.tight_layout()

In [ ]:
print(f"Number of cells: {len(good_cells)}")
good_cells["spot_count"] = 0
for cell_uid, count in spots_df["cell_uid"].value_counts().items():
    good_cells.loc[good_cells["cell_uid"] == cell_uid, "spot_count"] = count


fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].hist(good_cells["spot_count"], bins=np.arange(40) - 0.5)
perc, bins, bars = axes[1].hist(
    good_cells["spot_count"], bins=np.arange(50) - 0.5, cumulative=True, density=True
)


for n in [1, 3, 5]:
    lt_n = good_cells["spot_count"].lt(n).sum()
    print(f"Cells with less than {n} spots: {lt_n}, {lt_n/len(good_cells)*100:.2f}%")
axes[0].set_xlabel("Barcode spots per cell")
axes[1].set_xlabel("Barcode spots per cell")
axes[0].set_ylabel("# of Cells")
axes[1].set_ylabel("Cumulative fraction")
axes[1].set_xlim(0, 20)
axes[1].set_ylim(0, 1)
axes[0].set_xlim(-0.5, 40)
fig.tight_layout()

In [ ]:
good_cells["spot_count"].describe().round(1)

In [ ]:
no_spot = good_cells[good_cells["spot_count"] == 9]

# select 16 randomly
no_spot = no_spot.sample(16)

fig, axes = plt.subplots(4, 4, figsize=(20, 20))
for iax, (i, row) in enumerate(no_spot.iterrows()):
    ax = axes.ravel()[iax]
    xl = row["centroid-1"] + np.array([-100, 100])
    yl = row["centroid-0"] + np.array([-100, 100])
    xl = np.clip(xl, 0, rabies_stack.shape[1] - 1).astype(int)
    yl = np.clip(yl, 0, rabies_stack.shape[0] - 1).astype(int)
    data = issp.vis.utils.get_stack_part(rabies_stack[..., row["roi"] - 1], xl, yl)
    labels = issp.vis.utils.get_stack_part(labeled_images[..., row["roi"] - 1], xl, yl)
    rgb = issp.vis.to_rgb(data, colors=colors, vmin=[0, 0], vmax=vmaxs)
    ax.imshow(rgb)
    ax.contour(labels, levels=np.arange(labels.max()) + 0.5, colors="k", linewidths=0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"Cell {row['cell_uid']} - {int(row['area'])}")
fig.tight_layout()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
import matplotlib.pyplot as plt

save_path = "/nemo/project/proj-znamenp-barseq/processed/becalia_rabies_barseq/BRAC8780.3f/analysis/"
os.makedirs(save_path, exist_ok=True)

# Get list of all ROIs present in your DataFrame
all_rois = sorted(good_cells["roi"].unique())
no_spot = good_cells[good_cells["spot_count"] <= 2]

for roi_of_interest in all_rois:
    # Gather all cells in this ROI (both no_spot and not)
    roi_good_cells = good_cells[good_cells["roi"] == roi_of_interest]
    # Also find all no_spot cells in this ROI
    roi_no_spot_cells = no_spot[no_spot["roi"] == roi_of_interest]

    # If there are no cells in this ROI for some reason (unlikely),
    # skip this iteration
    if roi_good_cells.empty:
        continue

    # 1) Compute a bounding box around all cells in the ROI (with some margin)
    margin = 200
    min_y = int(roi_good_cells["centroid-0"].min() - margin)
    max_y = int(roi_good_cells["centroid-0"].max() + margin)
    min_x = int(roi_good_cells["centroid-1"].min() - margin)
    max_x = int(roi_good_cells["centroid-1"].max() + margin)

    # Clamp the bounding box to the image dimensions
    min_y = np.clip(min_y, 0, rabies_stack.shape[0] - 1)
    max_y = np.clip(max_y, 0, rabies_stack.shape[0] - 1)
    min_x = np.clip(min_x, 0, rabies_stack.shape[1] - 1)
    max_x = np.clip(max_x, 0, rabies_stack.shape[1] - 1)

    # 2) Extract the data and label slices for that bounding box
    data = issp.vis.utils.get_stack_part(
        rabies_stack[..., roi_of_interest - 1],
        [min_x, max_x],
        [min_y, max_y],
    )
    labels = issp.vis.utils.get_stack_part(
        labeled_images[..., roi_of_interest - 1],
        [min_x, max_x],
        [min_y, max_y],
    )

    # 3) Convert to RGB for display
    rgb = issp.vis.to_rgb(data, colors=colors, vmin=[0, 0], vmax=vmaxs)

    # 4) Plot
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(rgb)

    # (a) Outline *all* cells in black
    ax.contour(labels, levels=np.arange(labels.max()) + 0.5, colors="k", linewidths=0.5)

    # (b) Highlight the no_spot cells in white, if any
    if not roi_no_spot_cells.empty:
        # Here, assume "label_id" corresponds to the integer labels in `labels`
        no_spot_label_ids = roi_no_spot_cells["label"].unique()
        for lbl_id in no_spot_label_ids:
            ax.contour(labels == lbl_id, levels=[0.5], colors="white", linewidths=1)

    # ax.set_xticks([])
    # ax.set_yticks([])
    ax.set_title(f"ROI {roi_of_interest}")
    out_file = os.path.join(save_path, f"ROI_{roi_of_interest}.png")
    plt.savefig(out_file, dpi=150, bbox_inches="tight")
    plt.close(fig)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# ----------------------
# USER-DEFINED VARIABLES
# ----------------------
roi_of_interest = 5
min_y, max_y = 450, 1100
min_x, max_x = 400, 1000
colors = [(1, 0, 0), (0, 1, 1)]
vmaxs = [800, 200]

no_spot = good_cells[good_cells["spot_count"] <= 2]

save_path = "/nemo/project/proj-znamenp-barseq/processed/becalia_rabies_barseq/BRAC8780.3f/analysis/"
os.makedirs(save_path, exist_ok=True)

# Filter cells for ROI 5
roi_5_good_cells = good_cells[good_cells["roi"] == roi_of_interest]
roi_5_no_spot_cells = no_spot[no_spot["roi"] == roi_of_interest]

# Extract the data and label slices for the fixed bounding box
data = issp.vis.utils.get_stack_part(
    rabies_stack[..., roi_of_interest - 1],
    [min_x, max_x],
    [min_y, max_y],
)
labels = issp.vis.utils.get_stack_part(
    labeled_images[..., roi_of_interest - 1],
    [min_x, max_x],
    [min_y, max_y],
)

# Convert to RGB for display
rgb = issp.vis.to_rgb(data, colors=colors, vmin=[0, 0], vmax=vmaxs)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb)

# (a) Outline all cells in black
ax.contour(labels, levels=np.arange(labels.max()) + 0.5, colors="k", linewidths=0.5)

# (b) Highlight the no_spot cells in white
if not roi_5_no_spot_cells.empty:
    no_spot_label_ids = roi_5_no_spot_cells["label"].unique()
    for lbl_id in no_spot_label_ids:
        ax.contour(labels == lbl_id, levels=[0.5], colors="white", linewidths=2)

ax.set_title(f"ROI {roi_of_interest} (y,x from 500 to 1000)")
# out_file = os.path.join(save_path, f"ROI_{roi_of_interest}_fixed_bbox.png")
# plt.savefig(out_file, dpi=150, bbox_inches="tight")
# plt.close(fig)
plt.show()